<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/MLflow-4-LogSignature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [4]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/iris.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, stratify=y) # 80% treino e 20% teste

# *Log Model Signature*

In [11]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import ModelSignature, infer_signature
from mlflow.types.schema import Schema,ColSpec

exp1 = mlflow.set_experiment(experiment_name="experimento_logsignature")

mlflow.start_run(run_name="Run1.1",experiment_id=exp1.experiment_id)
model = tree.DecisionTreeClassifier(criterion="entropy",max_depth=2)
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.sklearn.autolog(
    log_input_examples=False,
    log_model_signatures=False,
    log_models=False)

input_data = [
    {"name":"SepalLengthCm", "type":"double"},
    {"name":"SepalWidthCm", "type":"double"},
    {"name":"PetalLengthCm", "type":"double"},
    {"name":"PetalWidthCm", "type":"double"}
]

output_data = [{"name":"Species","type":"integer"}]

input_schema = Schema([ColSpec(col["type"], col['name']) for col in input_data])
output_schema = Schema([ColSpec(col['type'], col['name']) for col in output_data])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

#mlflow.sklearn.log_model(model, "treemodel", signature=signature, input_example=input_example)
mlflow.sklearn.log_model(model, "treemodel", signature=signature)

mlflow.end_run()

get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

# Uso do NGROK

O Ngrok é uma ferramenta que permite criar um túnel seguro para conectar um servidor local a um servidor remoto, tornando possível expor uma aplicação local para a Internet. Essa ferramenta é muito útil para desenvolvedores que precisam testar suas aplicações localmente antes de colocá-las em produção.

In [10]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2fbGojiXWPL7flUUKztsxbdTdrP_3DXnMDuQVRd5G5n6Fhr6A"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Endereço do MLflow:", ngrok_tunnel.public_url)

Endereço do MLflow: https://2c15-35-234-20-191.ngrok-free.app
